In [4]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")
URL = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000001&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"
response = requests.get(URL)
results= response.json()
results.keys() 


dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])